In [1]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
ld_dim=4
T=10
# load dataset
mnist_images0 = np.load(f'Data/dataset_ld_{ld_dim}_0.npy')
mnist_images1 = np.load(f'Data/dataset_ld_{ld_dim}_1.npy')

mnist_images =np.concatenate((mnist_images0, mnist_images1), axis = 0)
print(np.shape(mnist_images))

np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)

# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,2])
min_array=np.array([0.1,0.05,0.01,0.08])
layer_array=np.array([5,10,20,50])
print(NUM_QUBITS)
print(T)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    #zero = torch.zeros(128, 256*7).to(device)

                    # concatenate the two tensors along the second dimension
                    #input_batch = torch.cat((input_batch, zero), dim=1)
                    #target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

(14780, 4)
2
10
(11,)
0
T=10 Epoch: 1/30 - Loss: 0.2361 b_loss=0.2361 - T: 1.99s/epoch ,tempo_previto=31.80 min0.1 nl5
1
T=10 Epoch: 2/30 - Loss: 0.0316 b_loss=0.0316 - T: 1.88s/epoch ,tempo_previto=30.10 min0.1 nl5
2
T=10 Epoch: 3/30 - Loss: 0.0203 b_loss=0.0203 - T: 1.85s/epoch ,tempo_previto=29.44 min0.1 nl5
3
T=10 Epoch: 4/30 - Loss: 0.0214 b_loss=0.0203 - T: 1.89s/epoch ,tempo_previto=30.10 min0.1 nl5
4
T=10 Epoch: 5/30 - Loss: 0.0195 b_loss=0.0195 - T: 1.89s/epoch ,tempo_previto=30.13 min0.1 nl5
5
T=10 Epoch: 6/30 - Loss: 0.0208 b_loss=0.0195 - T: 1.90s/epoch ,tempo_previto=30.18 min0.1 nl5
6
T=10 Epoch: 7/30 - Loss: 0.0207 b_loss=0.0195 - T: 1.90s/epoch ,tempo_previto=30.18 min0.1 nl5
7
T=10 Epoch: 8/30 - Loss: 0.0217 b_loss=0.0195 - T: 1.98s/epoch ,tempo_previto=31.38 min0.1 nl5
8
T=10 Epoch: 9/30 - Loss: 0.0219 b_loss=0.0195 - T: 1.92s/epoch ,tempo_previto=30.51 min0.1 nl5
9
T=10 Epoch: 10/30 - Loss: 0.0197 b_loss=0.0195 - T: 1.97s/epoch ,tempo_previto=31.25 min0.1 nl5
10
T=10

T=10 Epoch: 23/30 - Loss: 0.0002 b_loss=0.0002 - T: 2.00s/epoch ,tempo_previto=27.28 min0.01 nl5
23
T=10 Epoch: 24/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.89s/epoch ,tempo_previto=25.70 min0.01 nl5
24
T=10 Epoch: 25/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.93s/epoch ,tempo_previto=26.22 min0.01 nl5
25
T=10 Epoch: 26/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.92s/epoch ,tempo_previto=26.06 min0.01 nl5
26
T=10 Epoch: 27/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.87s/epoch ,tempo_previto=25.39 min0.01 nl5
27
T=10 Epoch: 28/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.91s/epoch ,tempo_previto=25.89 min0.01 nl5
28
T=10 Epoch: 29/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.88s/epoch ,tempo_previto=25.42 min0.01 nl5
29
T=10 Epoch: 30/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.89s/epoch ,tempo_previto=25.57 min0.01 nl5
(11,)
0
T=10 Epoch: 1/30 - Loss: 0.1153 b_loss=0.1153 - T: 1.92s/epoch ,tempo_previto=24.87 min0.08 nl5
1
T=10 Epoch: 2/30 - Loss: 0.0180 b_loss=0.0180 - T: 1.91s/epoch ,tempo_previto=24.78 min0.08 nl5
2

T=10 Epoch: 16/30 - Loss: 0.0974 b_loss=0.0743 - T: 1.87s/epoch ,tempo_previto=26.66 min0.05 nl5
16
T=10 Epoch: 17/30 - Loss: 0.0776 b_loss=0.0743 - T: 1.92s/epoch ,tempo_previto=27.32 min0.05 nl5
17
T=10 Epoch: 18/30 - Loss: 0.0739 b_loss=0.0739 - T: 1.88s/epoch ,tempo_previto=26.68 min0.05 nl5
18
T=10 Epoch: 19/30 - Loss: 0.0934 b_loss=0.0739 - T: 1.87s/epoch ,tempo_previto=26.55 min0.05 nl5
19
T=10 Epoch: 20/30 - Loss: 0.0807 b_loss=0.0739 - T: 1.99s/epoch ,tempo_previto=28.13 min0.05 nl5
20
T=10 Epoch: 21/30 - Loss: 0.0788 b_loss=0.0739 - T: 1.88s/epoch ,tempo_previto=26.56 min0.05 nl5
21
T=10 Epoch: 22/30 - Loss: 0.0911 b_loss=0.0739 - T: 1.88s/epoch ,tempo_previto=26.57 min0.05 nl5
22
T=10 Epoch: 23/30 - Loss: 0.0892 b_loss=0.0739 - T: 1.86s/epoch ,tempo_previto=26.19 min0.05 nl5
23
T=10 Epoch: 24/30 - Loss: 0.0843 b_loss=0.0739 - T: 2.03s/epoch ,tempo_previto=28.58 min0.05 nl5
24
T=10 Epoch: 25/30 - Loss: 0.0860 b_loss=0.0739 - T: 1.93s/epoch ,tempo_previto=27.21 min0.05 nl5
25


T=10 Epoch: 9/30 - Loss: 0.0172 b_loss=0.0172 - T: 3.28s/epoch ,tempo_previto=38.84 min0.1 nl10
9
T=10 Epoch: 10/30 - Loss: 0.0229 b_loss=0.0172 - T: 3.24s/epoch ,tempo_previto=38.38 min0.1 nl10
10
T=10 Epoch: 11/30 - Loss: 0.0210 b_loss=0.0172 - T: 3.24s/epoch ,tempo_previto=38.24 min0.1 nl10
11
T=10 Epoch: 12/30 - Loss: 0.0185 b_loss=0.0172 - T: 3.28s/epoch ,tempo_previto=38.67 min0.1 nl10
12
T=10 Epoch: 13/30 - Loss: 0.0181 b_loss=0.0172 - T: 3.23s/epoch ,tempo_previto=38.09 min0.1 nl10
13
T=10 Epoch: 14/30 - Loss: 0.0190 b_loss=0.0172 - T: 3.61s/epoch ,tempo_previto=42.51 min0.1 nl10
14
T=10 Epoch: 15/30 - Loss: 0.0224 b_loss=0.0172 - T: 3.30s/epoch ,tempo_previto=38.77 min0.1 nl10
15
T=10 Epoch: 16/30 - Loss: 0.0256 b_loss=0.0172 - T: 3.25s/epoch ,tempo_previto=38.18 min0.1 nl10
16
T=10 Epoch: 17/30 - Loss: 0.0225 b_loss=0.0172 - T: 3.21s/epoch ,tempo_previto=37.63 min0.1 nl10
17
T=10 Epoch: 18/30 - Loss: 0.0236 b_loss=0.0172 - T: 3.30s/epoch ,tempo_previto=38.64 min0.1 nl10
18
T=

T=10 Epoch: 1/30 - Loss: 0.0146 b_loss=0.0146 - T: 3.52s/epoch ,tempo_previto=31.66 min0.08 nl10
1
T=10 Epoch: 2/30 - Loss: 0.0140 b_loss=0.0140 - T: 3.29s/epoch ,tempo_previto=29.49 min0.08 nl10
2
T=10 Epoch: 3/30 - Loss: 0.0129 b_loss=0.0129 - T: 3.78s/epoch ,tempo_previto=33.81 min0.08 nl10
3
T=10 Epoch: 4/30 - Loss: 0.0124 b_loss=0.0124 - T: 3.45s/epoch ,tempo_previto=30.78 min0.08 nl10
4
T=10 Epoch: 5/30 - Loss: 0.0117 b_loss=0.0117 - T: 3.24s/epoch ,tempo_previto=28.88 min0.08 nl10
5
T=10 Epoch: 6/30 - Loss: 0.0126 b_loss=0.0117 - T: 3.31s/epoch ,tempo_previto=29.48 min0.08 nl10
6
T=10 Epoch: 7/30 - Loss: 0.0142 b_loss=0.0117 - T: 3.28s/epoch ,tempo_previto=29.17 min0.08 nl10
7
T=10 Epoch: 8/30 - Loss: 0.0137 b_loss=0.0117 - T: 3.32s/epoch ,tempo_previto=29.45 min0.08 nl10
8
T=10 Epoch: 9/30 - Loss: 0.0127 b_loss=0.0117 - T: 3.28s/epoch ,tempo_previto=29.07 min0.08 nl10
9
T=10 Epoch: 10/30 - Loss: 0.0159 b_loss=0.0117 - T: 3.78s/epoch ,tempo_previto=33.41 min0.08 nl10
10
T=10 Epo

...

T=10 Epoch: 7/30 - Loss: 0.0133 b_loss=0.0130 - T: 6.01s/epoch ,tempo_previto=29.35 min0.08 nl20
7
T=10 Epoch: 8/30 - Loss: 0.0154 b_loss=0.0130 - T: 5.99s/epoch ,tempo_previto=29.17 min0.08 nl20
8
T=10 Epoch: 9/30 - Loss: 0.0138 b_loss=0.0130 - T: 6.02s/epoch ,tempo_previto=29.21 min0.08 nl20
9
T=10 Epoch: 10/30 - Loss: 0.0134 b_loss=0.0130 - T: 6.01s/epoch ,tempo_previto=29.04 min0.08 nl20
10
T=10 Epoch: 11/30 - Loss: 0.0139 b_loss=0.0130 - T: 6.49s/epoch ,tempo_previto=31.25 min0.08 nl20
11
T=10 Epoch: 12/30 - Loss: 0.0129 b_loss=0.0129 - T: 6.04s/epoch ,tempo_previto=28.98 min0.08 nl20
12
T=10 Epoch: 13/30 - Loss: 0.0135 b_loss=0.0129 - T: 5.97s/epoch ,tempo_previto=28.58 min0.08 nl20
13
T=10 Epoch: 14/30 - Loss: 0.0151 b_loss=0.0129 - T: 6.01s/epoch ,tempo_previto=28.67 min0.08 nl20
14
T=10 Epoch: 15/30 - Loss: 0.0140 b_loss=0.0129 - T: 5.96s/epoch ,tempo_previto=28.32 min0.08 nl20
15
T=10 Epoch: 16/30 - Loss: 0.0145 b_loss=0.0129 - T: 6.49s/epoch ,tempo_previto=30.70 min0.08 nl20

T=10 Epoch: 29/30 - Loss: 0.0772 b_loss=0.0735 - T: 6.01s/epoch ,tempo_previto=36.17 min0.05 nl20
29
T=10 Epoch: 30/30 - Loss: 0.0760 b_loss=0.0735 - T: 6.56s/epoch ,tempo_previto=39.33 min0.05 nl20
(11,)
0
T=10 Epoch: 1/30 - Loss: 0.0555 b_loss=0.0555 - T: 6.05s/epoch ,tempo_previto=33.19 min0.01 nl20
1
T=10 Epoch: 2/30 - Loss: 0.0542 b_loss=0.0542 - T: 6.04s/epoch ,tempo_previto=33.03 min0.01 nl20
2
T=10 Epoch: 3/30 - Loss: 0.0537 b_loss=0.0537 - T: 6.02s/epoch ,tempo_previto=32.79 min0.01 nl20
3
T=10 Epoch: 4/30 - Loss: 0.0543 b_loss=0.0537 - T: 6.03s/epoch ,tempo_previto=32.76 min0.01 nl20
4
T=10 Epoch: 5/30 - Loss: 0.0563 b_loss=0.0537 - T: 6.53s/epoch ,tempo_previto=35.38 min0.01 nl20
5
T=10 Epoch: 6/30 - Loss: 0.0527 b_loss=0.0527 - T: 5.99s/epoch ,tempo_previto=32.36 min0.01 nl20
6
T=10 Epoch: 7/30 - Loss: 0.0471 b_loss=0.0471 - T: 6.03s/epoch ,tempo_previto=32.44 min0.01 nl20
7
T=10 Epoch: 8/30 - Loss: 0.0554 b_loss=0.0471 - T: 6.01s/epoch ,tempo_previto=32.26 min0.01 nl20
8
T

T=10 Epoch: 21/30 - Loss: 0.0199 b_loss=0.0186 - T: 14.55s/epoch ,tempo_previto=53.11 min0.1 nl50
21
T=10 Epoch: 22/30 - Loss: 0.0208 b_loss=0.0186 - T: 14.55s/epoch ,tempo_previto=52.85 min0.1 nl50
22
T=10 Epoch: 23/30 - Loss: 0.0242 b_loss=0.0186 - T: 14.58s/epoch ,tempo_previto=52.74 min0.1 nl50
23
T=10 Epoch: 24/30 - Loss: 0.0200 b_loss=0.0186 - T: 14.36s/epoch ,tempo_previto=51.68 min0.1 nl50
24
T=10 Epoch: 25/30 - Loss: 0.0196 b_loss=0.0186 - T: 14.51s/epoch ,tempo_previto=51.99 min0.1 nl50
25
T=10 Epoch: 26/30 - Loss: 0.0222 b_loss=0.0186 - T: 14.56s/epoch ,tempo_previto=51.94 min0.1 nl50
26
T=10 Epoch: 27/30 - Loss: 0.0184 b_loss=0.0184 - T: 14.57s/epoch ,tempo_previto=51.72 min0.1 nl50
27
T=10 Epoch: 28/30 - Loss: 0.0186 b_loss=0.0184 - T: 14.58s/epoch ,tempo_previto=51.50 min0.1 nl50
28
T=10 Epoch: 29/30 - Loss: 0.0240 b_loss=0.0184 - T: 14.51s/epoch ,tempo_previto=51.03 min0.1 nl50
29
T=10 Epoch: 30/30 - Loss: 0.0184 b_loss=0.0184 - T: 14.56s/epoch ,tempo_previto=50.96 min0.

T=10 Epoch: 12/30 - Loss: 0.0143 b_loss=0.0116 - T: 14.54s/epoch ,tempo_previto=11.63 min0.08 nl50
12
T=10 Epoch: 13/30 - Loss: 0.0136 b_loss=0.0116 - T: 14.55s/epoch ,tempo_previto=11.40 min0.08 nl50
13
T=10 Epoch: 14/30 - Loss: 0.0150 b_loss=0.0116 - T: 14.48s/epoch ,tempo_previto=11.10 min0.08 nl50
14
T=10 Epoch: 15/30 - Loss: 0.0142 b_loss=0.0116 - T: 14.52s/epoch ,tempo_previto=10.89 min0.08 nl50
15
T=10 Epoch: 16/30 - Loss: 0.0135 b_loss=0.0116 - T: 14.65s/epoch ,tempo_previto=10.74 min0.08 nl50
16
T=10 Epoch: 17/30 - Loss: 0.0132 b_loss=0.0116 - T: 14.50s/epoch ,tempo_previto=10.39 min0.08 nl50
17
T=10 Epoch: 18/30 - Loss: 0.0135 b_loss=0.0116 - T: 14.53s/epoch ,tempo_previto=10.17 min0.08 nl50
18
T=10 Epoch: 19/30 - Loss: 0.0145 b_loss=0.0116 - T: 14.57s/epoch ,tempo_previto=9.96 min0.08 nl50
19
T=10 Epoch: 20/30 - Loss: 0.0136 b_loss=0.0116 - T: 14.60s/epoch ,tempo_previto=9.73 min0.08 nl50
20
T=10 Epoch: 21/30 - Loss: 0.0148 b_loss=0.0116 - T: 14.59s/epoch ,tempo_previto=9.48

T=10 Epoch: 3/30 - Loss: 0.0553 b_loss=0.0466 - T: 14.55s/epoch ,tempo_previto=21.09 min0.01 nl50
3
T=10 Epoch: 4/30 - Loss: 0.0575 b_loss=0.0466 - T: 14.59s/epoch ,tempo_previto=20.92 min0.01 nl50
4
T=10 Epoch: 5/30 - Loss: 0.0581 b_loss=0.0466 - T: 14.53s/epoch ,tempo_previto=20.58 min0.01 nl50
5
T=10 Epoch: 6/30 - Loss: 0.0567 b_loss=0.0466 - T: 14.59s/epoch ,tempo_previto=20.43 min0.01 nl50
6
T=10 Epoch: 7/30 - Loss: 0.0521 b_loss=0.0466 - T: 14.47s/epoch ,tempo_previto=20.02 min0.01 nl50
7
T=10 Epoch: 8/30 - Loss: 0.0507 b_loss=0.0466 - T: 14.62s/epoch ,tempo_previto=19.98 min0.01 nl50
8
T=10 Epoch: 9/30 - Loss: 0.0504 b_loss=0.0466 - T: 14.53s/epoch ,tempo_previto=19.61 min0.01 nl50
9
T=10 Epoch: 10/30 - Loss: 0.0502 b_loss=0.0466 - T: 14.58s/epoch ,tempo_previto=19.44 min0.01 nl50
10
T=10 Epoch: 11/30 - Loss: 0.0589 b_loss=0.0466 - T: 14.52s/epoch ,tempo_previto=19.11 min0.01 nl50
11
T=10 Epoch: 12/30 - Loss: 0.0607 b_loss=0.0466 - T: 14.52s/epoch ,tempo_previto=18.87 min0.01 nl